In [ ]:
# Подкючаем нужны нам библиотеки
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np


# Загрузка данных, считываем датасет для работы с ним
spot_df = pd.read_csv('/kaggle/input/musicss/dataset.csv')

In [ ]:
#Dropout - это техника регуляризации, которая помогает предотвратить переобучение нейронной сети. Вот основные причины его применения:

# 1) Предотвращает переобучение: Dropout случайным образом отключает некоторые нейроны во время обучения, что заставляет сеть учиться 
#более общим представлениям данных.

# 2) Улучшает обобщающую способность: Сеть становится менее зависимой от отдельных нейронов и учится работать с различными комбинациями 
#активированных нейронов.

# 3) Эффективно работает как ансамбль моделей: При каждой эпохе обучения создается "новая" модель, что эквивалентно усреднению 
#множества моделей.
spot_df.isnull().sum()

In [ ]:
t=0
for artists in spot_df['artists']:
    t=t+1
    if not isinstance(artists,str):
        print(artists,t)
spot_df.iloc[65900]

In [ ]:
spot_df = spot_df.dropna()
spot_df.isnull().sum()

In [ ]:
spot_df

In [ ]:
# track_id: идентификатор Spotify для трека.
# artists: Имена художников
# album_name: Название альбома.
# track_name: Название трека
# popularity: Оценка популярности от 0 до 100.
# duration_ms: длина трека в миллисекундах.
# explicit: есть ли в треке откровенные тексты.
# danceability, energy, key, loudness, mode, speechiness, 
# acousticness, instrumentalness, liveness, valence, tempo, 
# time_signature: различные музыкальные и вокальные характеристики.
# track_genre: Жанр трека.
# danceability: Danceability описывает, насколько трек подходит для танцев, основываясь на сочетании музыкальных элементов, включая темп, стабильность ритма, силу удара и общую регулярность. Значение 0.0 обозначает наименьшую танцабельность, а 1.0 - наибольшую.
# energy. Энергия - это мера от 0.0 до 1.0 и представляет собой воспринимаемую меру интенсивности и активности. Обычно энергичные треки звучат быстро, громко и шумно. Например, дэт-метал имеет высокую энергетику, в то время как прелюдии Баха оцениваются низко по этой шкале.
# key: Ключ, в котором находится трек. Целые числа соответствуют нотам, используя стандартную нотацию Pitch Class. Пример: 0 = С, 1 = C#/ДЬ, 2 = D и т.д. Если ключ не был определен, значение равно -1.
# loudness: Общая громкость трека в децибелах (dB)
# mode: Режим показывает модальность (мажор или минор) трека, тип шкалы, из которой взято его мелодическое содержание. Мажор обозначается 1, а минор - 0.
# speechiness: Speechiness определяет наличие произнесённых слов в треке. Чем более «речевым» является запись (например, ток-шоу, аудиокнига, поэзия), тем ближе значение атрибута к 1.0. Значения выше 0.66 описывают треки, которые, вероятно, состоят полностью из произнесённых слов. Значения между 0.33 и 0.66 описывают треки, которые могут содержать как музыку, так и речь, как в секциях, так и в слоях, включая такие случаи, как рэп. Значения ниже 0.33 скорее всего представляют музыку и Другие не речевые треки.
# acousticness: Мера уверенности от 0.0 до 1.0 в том, что трек является акустическим. 1.0 представляет собой высокую уверенность в том, что трек акустический.
# instrumentalness: Прогнозирует, содержит ли трек вокал. Звуки "Ооһ" и "ааһ" рассматриваются как инструментальные в этом контексте. Рэп или треки с произносимыми словами явно являются "вокальными". 


In [ ]:
# Зависимость популярности песни от её громкости
plt.figure(figsize=(10,6))
sns.regplot(data=spot_df, y='loudness', x='popularity', color='c').set(title='Зависимость популярности песни от её громкости')

In [ ]:
# spot_df['explicit'] = spot_df['explicit'].map({'False': 0, 'True': 1})
# Зависимость популярности песни от её продолжительности
spot_df['duration_min'] = spot_df['duration_ms'] / 60000
plt.figure(figsize=(5,7))
sns.regplot(data=spot_df, y='popularity', x='duration_min').set(title='Зависимость популярности песни от её продолжительности')


In [ ]:
spot_df

In [ ]:

#plt.figure(figsize=(8, 6))
#sns.histplot(data=spot_df, x='track_genre', hue='popularity', multiple='stack', bins=20)
#plt.title('Распределение по жанру')
#plt.show()

In [ ]:

# Выберите стиль графика sns.set_theme (style="whitegrid")
# Установка размера графика
plt.figure(figsize=(7, 5))
# Построение гистограммы для отображения распределения длительности песен (в минутах)
#'data['duration_ms'] / 60000' переводит длительность из миллисекунд в минуты.
# bins=30 задает количество интервалов (бинов) на гистограмме, что управляет детализацией распределения.
# kde=True добавляет линию плотности, показывая непрерывное распределение данных.
# color="skyblue" задает цвет для заполненной области гистограммы.
# line_kws={} используется для настройки параметров линии KDE: 'color': 'red' задает цвет линии,
# 'lw': 2 - ширину линии.
# alpha=0.6 задает прозрачность области гистограммы, чтобы лучше видеть пересечения.
sns.histplot(spot_df['duration_ms'] / 60000, bins=30, kde=True, color="skyblue",
line_kws={'color': 'red', 'lw': 2}, alpha=0.6)
# Настройка заголовка и меток
plt.title("Распределение длительности песен (в минутах)", fontsize=16)
plt.xlabel('Длительность (минуты)', fontsize=14)
plt.ylabel("Частота", fontsize=14)

# Увеличение шрифта для меток осей
plt.tick_params(axis='both', which='major', labelsize=12)
# Показать график
plt.show()


In [ ]:
sns.set_style(style='darkgrid')
plt.figure(figsize=(10,5))
popular = genre.sort_values('popularity', ascending=False).head(10)
sns.barplot(y = 'track_genre', x = 'popularity', spot_df = popular).set(title='Top 5 Genres by Popularity')

In [ ]:
sns.displot(data=spot_df,
            x='tempo',
            kind='hist',
            bins=np.arange(0,210, 10))


In [ ]:
sns.set_style(style='darkgrid')
plt.figure(figsize=(10,5))
popular = track_genre.sort_values('popularity', ascending=False).head(10)
sns.barplot(y = 'track_genre', x = 'popularity', spot_df = popular).set(title='Top 5 Genres by Popularity')

In [ ]:

# Используем value_counts() для подсчета количества треков в каждом жанре
# Метод value_counts() создает частотную таблицу, подсчитывая количество треков # для каждого уникального значения в столбце 'track_genre'.
top_genres = spot_df['track_genre'].value_counts().head(10).index
# Фильтруем данные, оставляя только треки из топ-10 жанров 
filtered_data = spot_df[spot_df['track_genre'].isin(top_genres)]
# Устанавливаем стиль графика 
sns.set_theme(style="whitegrid")
# Устанавливаем размер графика 
plt.figure(figsize=(7, 6))

sns.boxplot(data=filtered_data, x='track_genre', y='popularity', hue='track_genre', palette="pastel", linewidth=1.5)
# Настройка заголовка и меток
plt.title('Популярность топ-10 жанров', fontsize=18)
plt.xlabel('Жанры', fontsize=14)
plt.ylabel('Популярность', fontsize=14)
# Увеличиваем шрифт для меток по оси Х и выставляем текст вертикально
plt.xticks(rotation=90, fontsize=12)
# Добавление сетки для улучшения восприятия
# plt.grid() добавляет сетку на график, а axis='y', linestyle='--', linewidth=0.5 позволяет настроить # линию сетки по оси У.
plt.grid(axise='y', linestyle='--', linewidth=0.5)
# Увеличиваем шрифт для меток оси у
plt.tick_params(axis='y', labelsize=12)
# Показать график 
plt.tight_layout() 
plt.show()

In [ ]:
spot_df

In [ ]:
cm = spot_df.drop(['key','mode','explicit'], axis=1).corr(method = 'pearson')
plt.figure(figsize=(14,6))
map = sns.heatmap(cm, annot = True, fmt = '.1g', vmin=-1, vmax=1, center=0, cmap='inferno', linewidths=1, linecolor='Black')
map.set_title('Correlation Heatmap between Variable')
map.set_xticklabels(map.get_xticklabels(), rotation=90)

In [ ]:

#Для анализа сначала стоит посмотреть на матрицу корреляции. Она будет представлена в виде тепловой карты.
#Так как мы уже закодировали колонки, в которых есть строки, то будем считать корреляцию между теми колонками, которые не кодировались. Параметры для heatmap:

# fmt - формат надписей в ячейках. .2f означает, что после запятой будут идти 2 цифры.
# annot - добавляет значения в ячейки при True
# стар - устанавливает цветовую схему.
# ах - устанавливает оси для тепловой карты.
# Датафрейм с незакодированными строками
corr_data = spot_df.loc[:, ['popularity', 'duration_min', 'explicit', 'danceability', 'energy', 'key', 'loudness',
'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
'valence', 'tempo', 'time_signature']]
# Построение тепловой карты корреляции
corr_data_1 = corr_data.corr()
fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(data=corr_data_1, fmt='.2f', annot=True, cmap='coolwarm', ax=ax)
plt.title('Матрица корреляции')
plt.tight_layout()
plt.show()


In [ ]:
from copy import copy
# копируем начальный датасет
temp_data = copy(spot_df)
# Разделяем артистов
temp_data['artists'] = temp_data['artists'].str.split(';')
data_exploded = temp_data.explode('artists')
# Находим сумму популярности для каждого артиста

art_pop = data_exploded.groupby('artists')['popularity'].sum().reset_index()
# Находим количество треков у каждого артиста.
art_count = data_exploded['artists'].value_counts().reset_index()
# Формируем итоговый датасет.
art_pop_count = pd.merge(art_pop, art_count, on='artists')
art_pop_count['avg_popularity'] = art_pop_count['popularity']/ art_pop_count['count']
art_pop_count = art_pop_count.sort_values('avg_popularity', ascending=False).reset_index(drop=True)


In [ ]:
top_5_artists = art_pop_count.head()
top_5_artists

In [ ]:

plt.figure(figsize=(12, 6)), sns.set_style('whitegrid')
sns.barplot(x=top_5_artists['artists'], y=top_5_artists['avg_popularity'], color='#3c8eff', orient='v')
plt.title('Топ 5 артистов наибольшей средней популярностью')
plt.xlabel('Артист или группа')
plt.ylabel('Средняя популярность')
plt.tight_layout()
plt.show()